In [1]:
import qube
import glob
import importlib
import inspect
import os
import ast
from collections import defaultdict
from sklearn.base import BaseEstimator
from git import Repo, Remote
from itertools import zip_longest

%qubed


              .+-------+ 
            .' :     .'|   QUBE | Quantitative Backtesting Environment
           +-------+'  |  
           |   : * |   |   (c) 2022,  ver. 0.3.23
           |  ,+---|---+ 
           |.'     | .'    
           +-------+'    


In [2]:
def iter_funcs(class_node):
    decls = {}
    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)
            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    # print(f"{x} = {dx.value if dx else None}")
                    decls[x] = dx.value if dx and isinstance(dx, ast.Constant) else None
    # return dict(reversed(decls))
    return dict(reversed(list(decls.items())))

In [3]:
def flatten(decl_list):
    for d in decl_list:
        try:
            yield d.id
        except AttributeError:
            try:
                yield d.func.id
            except AttributeError:
                yield None

def is_decorated(decorator_list, decorator_name):
    for x in flatten(decorator_list):
        if x == decorator_name:
            return True
    return False

strats = defaultdict(list)
for file in glob.glob('**/*.py', recursive=True):
    name = os.path.splitext(os.path.basename(file))[0]
    # Ignore __ files
    if name.startswith("__"):
        continue

    with open(file, 'r') as f:
        src = f.read()

    try:
        class_node = ast.parse(src)
    except:
        continue

    nodes = [node for node in ast.walk(class_node) if isinstance(node, ast.ClassDef)]
    for n in nodes:
        if is_decorated(n.decorator_list, 'signal_generator'):
            strats[file].append((n.name, ast.get_docstring(n), iter_funcs(n)))

In [4]:
print(dict(strats))

{
    'glxy_s3/models/utils.py': [
        (
            'WalkForwardTest',
            'Walk forward test',
            {'estimator': None, 'train_period': 4, 'test_period': 1, 'units': 'W'}
        ),
        (
            'RollingVolatilityScaledSizer',
            None,
            {
                'estimator': None,
                'max_capital_in_risk': None,
                'vol_periods': None,
                'vol_factor': 1,
                'seasoned_timeframe': '1H',
                'train_period': 4,
                'test_period': 1,
                'units': 'W',
                'emergency_stop_loss': 0,
                'debug': False
            }
        )
    ],
    'glxy_s3/models/seasons/season0.py': [
        (
            'HappyHours',
            '"   Basic generator (no filtering)',
            {
                'capital': None,
                'threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursTopDogs',
            '"   Trade only those ones that showed good performance during training period',
            {
                'capital': None,
                'threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'passing_threshold': 0.1,
                'debug': False
            }
        ),
        (
            'HappyHoursTrendFollower',
            '"   Using simplest trend filter MA: trade longs only if price > MA',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursRSI',
            '"   Try open postions only when RSI\n"    Here important note: we use RSI 
calculated on open prices (not closes):\n"    that is why we do not shift rsi\'s values   
\n"\n"    | 27-Oct-2022: RSI filter uses close prices',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'rsi_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursIBS',
            '"   Filter on IBS values',
            {
                'capital': None,
                'threshold': None,
                'ibs_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursFDI',
            '"   Fractal Dimensions Index as filter for market regime',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'fdi_min': None,
                'fdi_max': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursRsiManhattan',
            '"   Mixed RSI filtered seasonality and trend filter',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'rsi_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'g_type': None,
                'g_tsi_atr_period': 100,
                'g_tsi_atr_smoother': 'kama',
                'g_tsi_scaling_factor': 12,
                'g_tii_H': 20,
                'g_tii_ti_period': 3,
                'g_tii_period': 3,
                'g_revert_entry_on_guard': False,
                'g_revert_entry_fraction': 0.5,
                'debug': False
            }
        ),
        (
            'HappyHoursManhattan',
            '"   Mixed seasonality and tren

In [ ]:
file = 'incubator/research/meanreverting/nimble/nimblers.py'
with open(file, 'r') as f:
    src = f.read()
    try:
        class_node = ast.parse(src)
    except:
        raise

    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)

            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    print(f"{x} = {dx.value if dx and isinstance(dx, ast.Constant) else None}")

In [9]:
from qube.booster.boosterai import Boo
Boo.fzf('Insaner').show()

>>> insane-trend-breakout.yml - incubator/data/experiments/insane/insane-trend-breakout.yml
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
 |-[0]	Insane-breakout-default                           	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

 |-[1]	Insane-breakout-opt-stage-1                       	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout - opt stage 1 | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

 |-[2]	Insane-breakout-big-range                         	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout - all option | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

>>>

	- actions: .run(<num>), .clean(<num>), .show(<num>), .load(<num>)

# Git control

In [76]:
r = Repo('Qube2exp')
r.head.commit.tree

<git.Tree "cb6e35add4828d2d3a44217c9df79d2e11d75285">

In [77]:
r.index.diff()

[]

In [32]:
git = r.git

In [79]:
git.add()

''

In [80]:
git.commit(m="Second commit")

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_25103/4037781616.py:1 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_25103/4037781616.py'                 │
│ /usr/local/envs/ira_p39/lib/python3.9/site-packages/git/cmd.py:638 in <lambda>            │
│                                                                                           │
│    635 │   │   :return: Callable object that will execute call _call_process with your ar │
│    636 │   │   if name[0] == '_':                                                         │
│    637 │   │   │   return LazyMixin.__getattr__(self, name)                               │
│ ❱  638 │   │   return lambda *args, **kwargs: self._call_process(name, *args, **kwargs)   │
│    639 │                                                                                  │
│    640 │   def set_persistent_git_options(self, **kwargs: Any) -> None:                   │
│    641 │   │   """Specify command line options to the git executable                      │
│                                                                                           │
│ /usr/local/envs/ira_p39/lib/python3.9/site-packages/git/cmd.py:1183 in _call_process      │
│                                                                                           │
│   1180 │   │   call.append(dashify(method))                                               │
│   1181 │   │   call.extend(args_list)                                                     │
│   1182 │   │                                                                              │
│ ❱ 1183 │   │   return self.execute(call, **exec_kwargs)                                   │
│   1184 │                                                                                  │
│   1185 │   def _parse_object_header(self, header_line: str) -> Tuple[str, str, int]:      │
│   1186 │   │   """                                                                        │
│                                                                                           │
│ /usr/local/envs/ira_p39/lib/python3.9/site-packages/git/cmd.py:983 in execute             │
│                                                                                           │
│    980 │   │   # END handle debug printing                                                │
│    981 │   │                                                                              │
│    982 │   │   if with_exceptions and status != 0:                                        │
│ ❱  983 │   │   │   raise GitCommandError(redacted_command, status, stderr_value, stdout_v │
│    984 │   │                                                                              │
│    985 │   │   if isinstance(stdout_value, bytes) and stdout_as_string:  # could also be  │
│    986 │   │   │   stdout_value = safe_decode(stdout_value)                               │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git commit -m Second commit
  stdout: 'On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean'

In [68]:
hsh = r.commit('main').binsha.hex()
hsh

'dd13a1ab984c3933a92280f78ecc454f7f718d1d'

In [66]:
from qube.booster.boosterai import _generate_name_like_id
_generate_name_like_id(hsh, 3), _generate_name_like_id(hsh, 5, 5)

('Xoj', 'Gupuw')